In [247]:
import warnings
warnings.filterwarnings("ignore")

In [248]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()

In [10]:
import scipy
from scipy import signal
import IPython.display as ipd
import numpy as np
import librosa
#tf.compat.v1.enable_eager_execution()
print(tf.__version__)
print(tf.executing_eagerly())
import tensorflow as tf
from tensorflow.python.keras import backend as K

# adjust values to your needs
config = tf.compat.v1.ConfigProto( device_count = {'GPU': 1 , 'CPU': 8} )
sess = tf.compat.v1.Session(config=config) 
K.set_session(sess)

NameError: name 'tf' is not defined

In [287]:
#tf.debugging.set_log_device_placement(False)

# Create some tensors
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)

RuntimeError: Device placement logging must be set at program startup

In [251]:

windowLength = 256  #Window size is the amount of sample we apply windowing to
overlap      = round(0.25 * windowLength) # tell us how much we are shiffting to the right  during stft
ffTLength    = windowLength # the number of sample we consider it in each chunk of the signal when we segment the signal and when we pass it to stft( used for calculation )
fs           = 8000
numFeatures  = ffTLength//2 + 1
numSegments  = 8
window = scipy.signal.hamming(windowLength, sym=False)

In [252]:
def play(audio, sample_rate):
    ipd.display(ipd.Audio(data=audio, rate=sample_rate))  # load a local WAV file

In [12]:
def read_audio(filepath, sample_rate, normalize=True):
    audio, sr = librosa.load(filepath, sr=sample_rate)
    if normalize is True:
        div_fac = 1 / np.max(np.abs(audio)) / 3.0
        audio = audio * div_fac
        # audio = librosa.util.normalize(audio)
    return audio, sr

In [254]:

class FeatureExtractor:
    def __init__(self, audio, *, windowLength, overlap, sample_rate):
        self.audio = audio
        self.ffT_length = windowLength
        self.window_length = windowLength
        self.overlap = overlap
        self.sample_rate = sample_rate
        self.window = scipy.signal.hamming(self.window_length, sym=False)

    def get_stft_spectrogram(self):
        return librosa.stft(self.audio, n_fft=self.ffT_length, win_length=self.window_length, hop_length=self.overlap,
                            window=self.window, center=True)

    def get_audio_from_stft_spectrogram(self, stft_features):
        return librosa.istft(stft_features, win_length=self.window_length, hop_length=self.overlap,
                             window=self.window, center=True)

In [255]:
def l2_norm(vector):
    return np.square(vector)

def SDR(denoised, cleaned, eps=1e-7): # Signal to Distortion Ratio
    a = l2_norm(denoised)
    b = l2_norm(denoised - cleaned)
    a_b = a / b
    return np.mean(10 * np.log10(a_b + eps))

In [293]:
#620 gb
#model=tf.keras.models.load_model('F:\\ahmed\\Desktop\\noise reduction\\CNN\\Models\\audio620gb\\weights.03-0.14.h5')
#220gb
model=tf.keras.models.load_model('F:\\ahmed\\Desktop\\noise reduction\\CNN\\Models\\2\\Model 1st_epoch.h5')

In [257]:
noisyAudio,_=read_audio("F:\\ahmed\\Desktop\\noise reduction\\evaluation\\noisy3.wav",sample_rate=fs)

In [258]:
noiseAudioFeatureExtractor = FeatureExtractor(noisyAudio, windowLength=windowLength, overlap=overlap, sample_rate=fs)
noise_stft_features = noiseAudioFeatureExtractor.get_stft_spectrogram()
noisyPhase = np.angle(noise_stft_features)
print(noisyPhase.shape)
noise_stft_features = np.abs(noise_stft_features)
mean = np.mean(noise_stft_features)
std = np.std(noise_stft_features)
noise_stft_features = (noise_stft_features - mean) / std

(129, 410)


In [294]:
def prepare_input_features(stft_features):
    # Phase Aware Scaling: To avoid extreme differences (more than
    # 45 degree) between the noisy and clean phase, the clean spectral magnitude was encoded as similar to [21]:
    noisySTFT = np.concatenate([stft_features[:,0:numSegments-1], stft_features], axis=1)
    stftSegments = np.zeros((numFeatures, numSegments , noisySTFT.shape[1] - numSegments + 1))

    for index in range(noisySTFT.shape[1] - numSegments + 1):
        stftSegments[:,:,index] = noisySTFT[:,index:index + numSegments]
    return stftSegments

In [295]:
def revert_features_to_audio(features, phase, cleanMean=None, cleanStd=None):
    # scale the outpus back to the original range
    if cleanMean and cleanStd:
        features = cleanStd * features + cleanMean

    phase = np.transpose(phase, (1, 0))
    features = np.squeeze(features)

    # features = librosa.db_to_power(features)
    features = features * np.exp(1j * phase)  # that fixes the abs() ope previously done

    features = np.transpose(features, (1, 0))
    return noiseAudioFeatureExtractor.get_audio_from_stft_spectrogram(features)

In [296]:
predictors = prepare_input_features(noise_stft_features)

In [297]:
predictors = np.reshape(predictors, (predictors.shape[0], predictors.shape[1], 1, predictors.shape[2]))
predictors = np.transpose(predictors, (3, 0, 1, 2)).astype(np.float32)
print('predictors.shape:', predictors.shape)

predictors.shape: (261, 129, 8, 1)


In [298]:
STFTFullyConvolutional = model.predict(predictors)
print(STFTFullyConvolutional.shape)

Executing op __inference_keras_scratch_graph_46154 in device <unspecified>
Executing op __inference_keras_scratch_graph_46154 in device <unspecified>
(261, 129, 1, 1)


In [299]:
play(noisyAudio,fs)

In [300]:
denoisedAudioFullyConvolutional = revert_features_to_audio(STFTFullyConvolutional, noisyPhase, mean, std)
ipd.Audio(data=denoisedAudioFullyConvolutional, rate=fs) # load a local WAV file

In [266]:
cleanaudio,_=read_audio("F:\\ahmed\\Desktop\\noise reduction\\evaluation\\clean3.wav",sample_rate=fs)

In [23]:
!pip install pesq

  Created wheel for pesq: filename=pesq-0.0.2-cp37-cp37m-win_amd64.whl size=119882 sha256=b647980d8a6cd7b31197373019917d245c7287585e0d50d863c561cdd596c929
  Stored in directory: c:\users\ahmed\appdata\local\pip\cache\wheels\c3\34\04\86b0c6c0d3bf5d547542ce1171dfe9e801d65a81c2eb02fe8c
Successfully built pesq


In [267]:
from scipy.io import wavfile
from pesq import pesq

#rate, ref = wavfile.read("./audio/speech.wav")
#rate, deg = wavfile.read("./audio/speech_bab_0dB.wav")

print(pesq(16000, cleanaudio,denoisedAudioFullyConvolutional, 'wb')) #wide band for sample rate =16000
print(pesq(fs,cleanaudio,denoisedAudioFullyConvolutional, 'nb')) # narrow band for sample rate =8000

2.3409738540649414
2.810666799545288


In [301]:
evaluation="F:\\ahmed\\Desktop\\noise reduction\\evaluation\\"
PESQ=[]
for i in range(1,52):
    noisyAudio,_ = read_audio(evaluation+"noisy"+str(i)+".wav",sample_rate=fs)
    cleanaudio,_=read_audio(evaluation+"clean"+str(i)+".wav",sample_rate=fs)
    noiseAudioFeatureExtractor = FeatureExtractor(noisyAudio, windowLength=windowLength, overlap=overlap, sample_rate=fs)
    noise_stft_features = noiseAudioFeatureExtractor.get_stft_spectrogram()
    noisyPhase = np.angle(noise_stft_features)
    noise_stft_features = np.abs(noise_stft_features)
    mean = np.mean(noise_stft_features)
    std = np.std(noise_stft_features)
    noise_stft_features = (noise_stft_features - mean) / std
    predictors = prepare_input_features(noise_stft_features)
    predictors = np.reshape(predictors, (predictors.shape[0], predictors.shape[1], 1, predictors.shape[2]))
    predictors = np.transpose(predictors, (3, 0, 1, 2)).astype(np.float32)
    STFTFullyConvolutional = model.predict(predictors)
    output = revert_features_to_audio(STFTFullyConvolutional, noisyPhase, mean, std)
    PESQ.append(pesq(fs,cleanaudio,output, 'nb'))

Executing op __inference_keras_scratch_graph_46154 in device <unspecified>
Executing op __inference_keras_scratch_graph_46154 in device <unspecified>
Executing op __inference_keras_scratch_graph_46154 in device <unspecified>
Executing op __inference_keras_scratch_graph_46154 in device <unspecified>
Executing op __inference_keras_scratch_graph_46154 in device <unspecified>
Executing op __inference_keras_scratch_graph_46154 in device <unspecified>
Executing op __inference_keras_scratch_graph_46154 in device <unspecified>
Executing op __inference_keras_scratch_graph_46154 in device <unspecified>
Executing op __inference_keras_scratch_graph_46154 in device <unspecified>
Executing op __inference_keras_scratch_graph_46154 in device <unspecified>
Executing op __inference_keras_scratch_graph_46154 in device <unspecified>
Executing op __inference_keras_scratch_graph_46154 in device <unspecified>
Executing op __inference_keras_scratch_graph_46154 in device <unspecified>
Executing op __inference_

In [302]:
sum(PESQ)/len(PESQ)

2.341785846971998

In [292]:
PESQ

[1.7055991888046265,
 2.255990743637085,
 2.810666799545288,
 2.7943615913391113,
 2.658108711242676,
 2.4904325008392334,
 2.508310556411743,
 2.526463031768799,
 1.9424712657928467,
 2.337850332260132,
 2.3707706928253174,
 2.794086217880249,
 2.386599063873291,
 1.6719448566436768,
 1.9064826965332031,
 2.246946096420288,
 2.276733160018921,
 2.59165620803833,
 2.6731014251708984,
 2.336765766143799,
 2.4903697967529297,
 2.3503828048706055,
 2.885896682739258,
 2.417714834213257,
 2.4401047229766846,
 2.0771892070770264,
 2.3023390769958496,
 2.044689416885376,
 2.311967611312866,
 2.4761786460876465,
 2.391352891921997,
 2.3099818229675293,
 2.5244979858398438,
 2.239647150039673,
 2.612506866455078,
 2.5475337505340576,
 1.9968359470367432,
 2.8257691860198975,
 2.0805509090423584,
 1.6570351123809814,
 2.439767837524414,
 2.569565773010254,
 2.130460023880005,
 1.6968902349472046,
 2.463757038116455,
 2.3208072185516357,
 2.1194074153900146,
 2.500234842300415,
 2.15069127082824

# Facebook Research 

In [1]:
# Copyright (c) Facebook, Inc. and its affiliates.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
# author: adiyoss

import argparse
from concurrent.futures import ProcessPoolExecutor
from collections import namedtuple
from pathlib import Path
from torch.nn import functional as F
import json
import logging
import os
import sys
import math
import torch
import torchaudio
from torch import nn
from torch.utils.data.distributed import DistributedSampler
from torch.utils.data import DataLoader, Subset
from torch.nn.parallel.distributed import DistributedDataParallel

import logging

import torch.hub



logger = logging.getLogger(__name__)


F:\anaconda3\envs\waveunet\lib\site-packages\torchaudio\extension\extension.py:13: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')


In [2]:
import math

import torch as th
from torch.nn import functional as F


def sinc(t):
    """sinc.
    :param t: the input tensor
    """
    return th.where(t == 0, th.tensor(1., device=t.device, dtype=t.dtype), th.sin(t) / t)


def kernel_upsample2(zeros=56):
    """kernel_upsample2.
    """
    win = th.hann_window(4 * zeros + 1, periodic=False)
    winodd = win[1::2]
    t = th.linspace(-zeros + 0.5, zeros - 0.5, 2 * zeros)
    t *= math.pi
    kernel = (sinc(t) * winodd).view(1, 1, -1)
    return kernel


def upsample2(x, zeros=56):
    """
    Upsampling the input by 2 using sinc interpolation.
    Smith, Julius, and Phil Gossett. "A flexible sampling-rate conversion method."
    ICASSP'84. IEEE International Conference on Acoustics, Speech, and Signal Processing.
    Vol. 9. IEEE, 1984.
    """
    *other, time = x.shape
    kernel = kernel_upsample2(zeros).to(x)
    out = F.conv1d(x.view(-1, 1, time), kernel, padding=zeros)[..., 1:].view(*other, time)
    y = th.stack([x, out], dim=-1)
    return y.view(*other, -1)


def kernel_downsample2(zeros=56):
    """kernel_downsample2.
    """
    win = th.hann_window(4 * zeros + 1, periodic=False)
    winodd = win[1::2]
    t = th.linspace(-zeros + 0.5, zeros - 0.5, 2 * zeros)
    t.mul_(math.pi)
    kernel = (sinc(t) * winodd).view(1, 1, -1)
    return kernel


def downsample2(x, zeros=56):
    """
    Downsampling the input by 2 using sinc interpolation.
    Smith, Julius, and Phil Gossett. "A flexible sampling-rate conversion method."
    ICASSP'84. IEEE International Conference on Acoustics, Speech, and Signal Processing.
    Vol. 9. IEEE, 1984.
    """
    if x.shape[-1] % 2 != 0:
        x = F.pad(x, (0, 1))
    xeven = x[..., ::2]
    xodd = x[..., 1::2]
    *other, time = xodd.shape
    kernel = kernel_downsample2(zeros).to(x)
    out = xeven + F.conv1d(xodd.view(-1, 1, time), kernel, padding=zeros)[..., :-1].view(
        *other, time)
    return out.view(*other, -1).mul(0.5)

In [3]:
class BLSTM(nn.Module):
    def __init__(self, dim, layers=2, bi=True):
        super().__init__()
        klass = nn.LSTM
        self.lstm = klass(bidirectional=bi, num_layers=layers, hidden_size=dim, input_size=dim)
        self.linear = None
        if bi:
            self.linear = nn.Linear(2 * dim, dim)

    def forward(self, x, hidden=None):
        x, hidden = self.lstm(x, hidden)
        if self.linear:
            x = self.linear(x)
        return x, hidden


def rescale_conv(conv, reference):
    std = conv.weight.std().detach()
    scale = (std / reference)**0.5
    conv.weight.data /= scale
    if conv.bias is not None:
        conv.bias.data /= scale


def rescale_module(module, reference):
    for sub in module.modules():
        if isinstance(sub, (nn.Conv1d, nn.ConvTranspose1d)):
            rescale_conv(sub, reference)
class Demucs(nn.Module):
    """
    Demucs speech enhancement model.
    Args:
        - chin (int): number of input channels.
        - chout (int): number of output channels.
        - hidden (int): number of initial hidden channels.
        - depth (int): number of layers.
        - kernel_size (int): kernel size for each layer.
        - stride (int): stride for each layer.
        - causal (bool): if false, uses BiLSTM instead of LSTM.
        - resample (int): amount of resampling to apply to the input/output.
            Can be one of 1, 2 or 4.
        - growth (float): number of channels is multiplied by this for every layer.
        - max_hidden (int): maximum number of channels. Can be useful to
            control the size/speed of the model.
        - normalize (bool): if true, normalize the input.
        - glu (bool): if true uses GLU instead of ReLU in 1x1 convolutions.
        - rescale (float): controls custom weight initialization.
            See https://arxiv.org/abs/1911.13254.
        - floor (float): stability flooring when normalizing.
    """
    def __init__(self,
                 chin=1,
                 chout=1,
                 hidden=64,
                 depth=5,
                 kernel_size=8,
                 stride=4,
                 causal=True,
                 resample=4,
                 growth=2,
                 max_hidden=10_000,
                 normalize=True,
                 glu=True,
                 rescale=0.1,
                 floor=1e-3):

        super().__init__()
        if resample not in [1, 2, 4]:
            raise ValueError("Resample should be 1, 2 or 4.")

        self.chin = chin
        self.chout = chout
        self.hidden = hidden
        self.depth = depth
        self.kernel_size = kernel_size
        self.stride = stride
        self.causal = causal
        self.floor = floor
        self.resample = resample
        self.normalize = normalize

        self.encoder = nn.ModuleList()
        self.decoder = nn.ModuleList()
        activation = nn.GLU(1) if glu else nn.ReLU()
        ch_scale = 2 if glu else 1

        for index in range(depth):
            encode = []
            encode += [
                nn.Conv1d(chin, hidden, kernel_size, stride),
                nn.ReLU(),
                nn.Conv1d(hidden, hidden * ch_scale, 1), activation,
            ]
            self.encoder.append(nn.Sequential(*encode))

            decode = []
            decode += [
                nn.Conv1d(hidden, ch_scale * hidden, 1), activation,
                nn.ConvTranspose1d(hidden, chout, kernel_size, stride),
            ]
            if index > 0:
                decode.append(nn.ReLU())
            self.decoder.insert(0, nn.Sequential(*decode))
            chout = hidden
            chin = hidden
            hidden = min(int(growth * hidden), max_hidden)

        self.lstm = BLSTM(chin, bi=not causal)
        if rescale:
            rescale_module(self, reference=rescale)

    def valid_length(self, length):
        """
        Return the nearest valid length to use with the model so that
        there is no time steps left over in a convolutions, e.g. for all
        layers, size of the input - kernel_size % stride = 0.
        If the mixture has a valid length, the estimated sources
        will have exactly the same length.
        """
        length = math.ceil(length * self.resample)
        for idx in range(self.depth):
            length = math.ceil((length - self.kernel_size) / self.stride) + 1
            length = max(length, 1)
        for idx in range(self.depth):
            length = (length - 1) * self.stride + self.kernel_size
        length = int(math.ceil(length / self.resample))
        return int(length)

    
    def total_stride(self):
        return self.stride ** self.depth // self.resample

    def forward(self, mix):
        if mix.dim() == 2:
            mix = mix.unsqueeze(1)

        if self.normalize:
            mono = mix.mean(dim=1, keepdim=True)
            std = mono.std(dim=-1, keepdim=True)
            mix = mix / (self.floor + std)
        else:
            std = 1
        length = mix.shape[-1]
        x = mix
        x = F.pad(x, (0, self.valid_length(length) - length))
        if self.resample == 2:
            x = upsample2(x)
        elif self.resample == 4:
            x = upsample2(x)
            x = upsample2(x)
        skips = []
        for encode in self.encoder:
            x = encode(x)
            skips.append(x)
        x = x.permute(2, 0, 1)
        x, _ = self.lstm(x)
        x = x.permute(1, 2, 0)
        for decode in self.decoder:
            skip = skips.pop(-1)
            x = x + skip[..., :x.shape[-1]]
            x = decode(x)
        if self.resample == 2:
            x = downsample2(x)
        elif self.resample == 4:
            x = downsample2(x)
            x = downsample2(x)

        x = x[..., :length]
        return std * x


In [4]:
def get_model():
  DNS_64_URL= "https://dl.fbaipublicfiles.com/adiyoss/denoiser/dns64-a7761ff99a7d5bb6.th"
  model = Demucs()
  state_dict = torch.hub.load_state_dict_from_url(DNS_64_URL, map_location='cpu')
  model.load_state_dict(state_dict)
  #model = _demucs(DNS_64_URL, hidden=64)
  return model


In [5]:
model=get_model()


In [6]:
model.eval()

Demucs(
  (encoder): ModuleList(
    (0): Sequential(
      (0): Conv1d(1, 64, kernel_size=(8,), stride=(4,))
      (1): ReLU()
      (2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (3): GLU(dim=1)
    )
    (1): Sequential(
      (0): Conv1d(64, 128, kernel_size=(8,), stride=(4,))
      (1): ReLU()
      (2): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
      (3): GLU(dim=1)
    )
    (2): Sequential(
      (0): Conv1d(128, 256, kernel_size=(8,), stride=(4,))
      (1): ReLU()
      (2): Conv1d(256, 512, kernel_size=(1,), stride=(1,))
      (3): GLU(dim=1)
    )
    (3): Sequential(
      (0): Conv1d(256, 512, kernel_size=(8,), stride=(4,))
      (1): ReLU()
      (2): Conv1d(512, 1024, kernel_size=(1,), stride=(1,))
      (3): GLU(dim=1)
    )
    (4): Sequential(
      (0): Conv1d(512, 1024, kernel_size=(8,), stride=(4,))
      (1): ReLU()
      (2): Conv1d(1024, 2048, kernel_size=(1,), stride=(1,))
      (3): GLU(dim=1)
    )
  )
  (decoder): ModuleList(
    (0): Seque

In [7]:
torch.save(model,"F:\\ahmed\\Desktop\\noise reduction\\facebookresearch\\denoiserfb.pt" )

In [8]:
model=torch.load("F:\\ahmed\\Desktop\\noise reduction\\facebookresearch\\denoiserfb.pt")

In [280]:
evaluation="F:\\ahmed\\Desktop\\noise reduction\\evaluation\\"
PESQ=[]
for i in range(1,52):
    inp,_ = read_audio(evaluation+"noisy"+str(i)+".wav",sample_rate=fs)
    cleanaudio,_=read_audio(evaluation+"clean"+str(i)+".wav",sample_rate=fs)
    inp=np.expand_dims(inp, axis=0)
    inp=torch.from_numpy(inp)
    output=model(inp)
    output=torch.squeeze(output)
    PESQ.append(pesq(fs,cleanaudio,output.detach().numpy(), 'nb'))
    
    

In [196]:
len(PESQ)

15

In [281]:
PESQ

[1.977802038192749,
 2.6957833766937256,
 3.6690473556518555,
 3.426020622253418,
 3.5820789337158203,
 3.309804677963257,
 2.9783217906951904,
 2.661221504211426,
 2.2237188816070557,
 2.6353814601898193,
 2.3941991329193115,
 2.950261116027832,
 1.227853775024414,
 1.3753548860549927,
 1.4199007749557495,
 2.604166269302368,
 3.5847315788269043,
 2.9951906204223633,
 2.4529197216033936,
 1.3366495370864868,
 4.039168357849121,
 3.1174697875976562,
 3.817420721054077,
 3.265660524368286,
 2.7284443378448486,
 1.8177093267440796,
 3.081399917602539,
 1.7708287239074707,
 3.9313390254974365,
 3.2045841217041016,
 3.2984983921051025,
 2.303117513656616,
 3.7530391216278076,
 3.6932146549224854,
 2.988583564758301,
 3.007599115371704,
 3.6478846073150635,
 1.4845507144927979,
 1.4995486736297607,
 1.2054463624954224,
 3.974266767501831,
 1.5783846378326416,
 2.3847496509552,
 1.7080116271972656,
 3.715980052947998,
 2.8953442573547363,
 2.0927398204803467,
 2.3871026039123535,
 2.45803236

In [282]:
print(sum(PESQ)/len(PESQ))

2.729229415164274


In [339]:
inp,_ = read_audio("F:\\ahmed\\Desktop\\noise reduction\\facebookresearch\\noisy_test\\seventh noisy.wav",sample_rate=16000)

In [14]:
#inp, sr = torchaudio.load("F:\\ahmed\\Desktop\\noise reduction\\facebookresearch\\noisy_test\\fourth noisy.wav", frame_offset=0, num_frames=-1)
inp,_ = read_audio("F:\\ahmed\\Desktop\\noise reduction\\facebookresearch\\noisy_test\\fourth noisy.wav",sample_rate=16000)

In [15]:
inp=np.expand_dims(inp, axis=0)

In [16]:
inp

array([[ 0.00675651,  0.01141677,  0.01387637, ...,  0.00292408,
        -0.00013619, -0.00095892]], dtype=float32)

In [17]:
inp=torch.from_numpy(inp)

In [18]:
inp.shape

torch.Size([1, 26094])

In [19]:
output=model(inp)

In [345]:
#inp=torch.squeeze(inp)

In [20]:
output=torch.squeeze(output)

In [21]:
ipd.display(ipd.Audio(data=inp, rate=16000))
ipd.display(ipd.Audio(data=output.detach().numpy(), rate=16000))

In [246]:
print(pesq(16000, cleanaudio,output.detach().numpy(), 'wb')) #wide band for sample rate =16000
print(pesq(fs,cleanaudio,output.detach().numpy(), 'nb')) # narrow band for sample rate =8000

3.135559558868408
3.6690473556518555


In [348]:
scipy.io.wavfile.write("F:\\ahmed\\Desktop\\noise reduction\\facebookresearch\\noisy_test\\seventh enhanced.wav", 16000, output.detach().numpy())